# Bidirectional LSTM 



## Preparing data 

In [27]:
import requests
import torch
import torch.nn.functional as F 
import torchtext
import re
import unicodedata
MAX_LENGTH = 10
url="https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/"

train_en = [line.split() for line in requests.get(url + "train.en").text.splitlines()]
train_vi = [line.split() for line in requests.get(url + "train.vi").text.splitlines()]

test_en = [line.split() for line in requests.get(url + "tst2013.en").text.splitlines()]
test_vi = [line.split() for line in requests.get(url + "tst2013.vi").text.splitlines()]

In [24]:
type(train_en)

list

In [25]:
print(len(train_en))
train_vi[0]

133317


['Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']

In [28]:
MODELNAME = "iwslt15-en-vi-rnn.model"
# EPOCH = 10
BATCHSIZE = 64 
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [29]:
def make_vocab(train_data, min_freq):
  vocab = {}
  for tokenlist in train_data:
    for token in tokenlist:
      if token not in vocab:
        vocab[token] = 0
      vocab[token] += 1
  vocablist = [('<unk>', 0), ('<pad>', 0), ('<cls>', 0), ('<eos>', 0)]
  vocabidx = {}
  for token, freq in vocab.items():
    if freq >= min_freq:
      idx = len(vocablist)
      vocablist.append((token, freq))
      vocabidx[token]=idx
  vocabidx['<unk>']=0
  vocabidx['<pad>']=1
  vocabidx['<cls>']=2
  vocabidx['<eos>']=3
  return vocablist, vocabidx

vocablist_en, vocabidx_en = make_vocab(train_en, 3)
vocablist_vi, vocabidx_vi = make_vocab(train_vi, 3)

print("vocab size en:", len(vocablist_en))
print("vocab size vi:", len(vocablist_vi))


vocab size en: 24420
vocab size vi: 10666


In [30]:
def preprocess(data, vocabidx):
  rr = []
  for tokenlist in data:
    tkl = ['<cls>']
    for token in tokenlist:
      tkl.append(token if token in vocabidx else '<unk>')
    tkl.append('<eos>')
    rr.append(tkl)
  return rr


train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)

for i in range(5):
  print(train_en_prep[i])
  print(train_vi_prep[i])
  print(test_en_prep[i])

['<cls>', 'Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
['<cls>', 'Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu', '<eos>']
['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>']
['<cls>', 'In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
['<cls>', 'Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu

In [31]:
train_data = list(zip(train_en_prep, train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))
test_data = list(zip(test_en_prep, test_en, test_vi))

for i in range(5):
  print(train_data[i])

for i in range(5):   
  print(test_data[i])

(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>'], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;'])
(['<cls>', 'And', 'I', 'was', 'very', 'proud', '.', '<eos>

In [32]:
def make_batch(data, batchsize):
  bb = []
  ben = []
  bvi = []
  for en, vi in data: 
    ben.append(en)
    bvi.append(vi)
    if len(ben) >= batchsize:
      bb.append((ben, bvi))
      ben = []
      bvi = []
  if len(ben) > 0:
    bb.append((ben, bvi))
  return bb

train_data = make_batch(train_data, BATCHSIZE)

for i in range(5):
  print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']

In [33]:
def padding_batch(b):
  maxlen = max([len(x) for x in b])
  for tkl in b:
    for i in range(maxlen - len(tkl)):
      tkl.append('<pad>')

def padding(bb):
  for ben, bvi in bb:
    padding_batch(ben)
    padding_batch(bvi)

padding(train_data)

for i in range(3):
  print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']

In [34]:
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi [token] for token in tokenlist] for tokenlist in bvi]) for ben, bvi in train_data]
test_data = [([vocabidx_en[token] for token in enprep], en, vi) for enprep, en, vi in test_data]

for i in range (3): 
  print(train_data[i]) 
for i in range(3): 
  print(test_data[i])

([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 

In [12]:
type(train_data)

list

## Build the model

### Model LSTM 40 epoch, bleu = 0.0959

In [40]:
class LSTM(torch.nn.Module):
  def __init__(self, vocablist_x, vocabidx_x, vocablist_y, vocabidx_y):
    super(LSTM, self).__init__()

    self.encemb = torch.nn.Embedding(len(vocablist_x), 256, padding_idx = vocabidx_x['<pad>'])
    self.dropout = torch.nn.Dropout(0.1)
    self.enclstm = torch.nn.LSTM(256,516,1,dropout=0.1, bidirectional=True)
    
    self.decemb = torch.nn.Embedding(len(vocablist_x), 256, padding_idx = vocabidx_y['<pad>'])
    self.declstm = torch.nn.LSTM(256,516,1,dropout=0.1, bidirectional=True)
    self.fc1 = torch.nn.Linear(2*516, 2*2*516 )
    self.fc2 = torch.nn.Linear(2*2*516, len(vocabidx_y))
  
  def forward(self,x):
    x, y = x[0], x[1]
    e_x = self.dropout(self.encemb(x))
    outenc,(h,c) = self.enclstm(e_x)
    n_y=y.shape[0]
    outputs = torch.zeros(n_y,BATCHSIZE,len(vocablist_vi)).to(DEVICE)
    loss = torch.tensor(0.,dtype=torch.float32).to(DEVICE)
    for i in range(n_y-1):
      input = y[i]
      input = input.unsqueeze(0)
      input = self.dropout(self.decemb(input))
      outdec, (h,c) = self.declstm(input,(h,c))
      dense_output = self.fc1(outdec)
      drop_output = self.dropout(dense_output)
      output = self.fc2(drop_output.squeeze(0))
      input = y[i+1]
      loss += F.cross_entropy(output, y[i+1])
    return loss

  def evaluate(self,x,vocablist_y,vocabidx_y):
    e_x = self.dropout(self.encemb(x))
    outenc,(h,c)=self.enclstm(e_x)
    
    y = torch.tensor([vocabidx_y['<cls>']]).to(DEVICE)
    pred=[]
    for i in range(30):
        input = y
        input = input.unsqueeze(0)
        input = self.dropout(self.decemb(input))
        outdec, (h,c) = self.declstm(input,(h,c))
        dense_output = self.fc1(outdec)
        drop_output = self.dropout(dense_output)
        output = self.fc2(drop_output.squeeze(0))
        pred_id = output.squeeze().argmax().item()
        if pred_id == vocabidx_y['<eos>']:
            break
        pred_y = vocablist_y[pred_id][0]
        pred.append(pred_y)
        y[0]=pred_id
        input=y
    return pred

### Model LTSM 20 epoch, bleu = 0.1

In [35]:
class LSTM(torch.nn.Module):
  def __init__(self, vocablist_x, vocabidx_x, vocablist_y, vocabidx_y):
    super(LSTM, self).__init__()

    self.encemb = torch.nn.Embedding(len(vocablist_x), 500, padding_idx = vocabidx_x['<pad>'])
    self.dropout = torch.nn.Dropout(0.2)
    self.enclstm = torch.nn.LSTM(500,512,2,dropout=0.2, bidirectional=True)
    
    self.decemb = torch.nn.Embedding(len(vocablist_x), 500, padding_idx = vocabidx_y['<pad>'])
    self.declstm = torch.nn.LSTM(500,512,2,dropout=0.2, bidirectional=True)
    self.decout = torch.nn.Linear(512*2, len(vocabidx_y))
    # self.fc1 = torch.nn.Linear(2*512, 2*2*512 )
    # self.fc2 = torch.nn.Linear(2*512, len(vocabidx_y))
  
  def forward(self,x):
    x, y = x[0], x[1]
    e_x = self.dropout(self.encemb(x))
    outenc,(h,c) = self.enclstm(e_x)
    n_y=y.shape[0]
    outputs = torch.zeros(n_y,BATCHSIZE,len(vocablist_vi)).to(DEVICE)
    loss = torch.tensor(0.,dtype=torch.float32).to(DEVICE)
    for i in range(n_y-1):
      input = y[i]
      input = input.unsqueeze(0)
      input = self.dropout(self.decemb(input))
      outdec, (h,c) = self.declstm(input,(h,c))
      # dense_output = self.fc1(outdec)
      # drop_output = self.dropout(dense_output)
      # output = self.fc2(drop_output.squeeze(0))
      output = self.decout(outdec.squeeze(0))
      input = y[i+1]
      loss += F.cross_entropy(output, y[i+1])
    return loss

  def evaluate(self,x,vocablist_y,vocabidx_y):
    e_x = self.dropout(self.encemb(x))
    outenc,(h,c)=self.enclstm(e_x)
    
    y = torch.tensor([vocabidx_y['<cls>']]).to(DEVICE)
    pred=[]
    for i in range(30):
        input = y
        input = input.unsqueeze(0)
        input = self.dropout(self.decemb(input))
        outdec, (h,c) = self.declstm(input,(h,c))
        # dense_output = self.fc1(outdec)
        # drop_output = self.dropout(dense_output)
        # output = self.fc2(drop_output.squeeze(0))
        output = self.decout(outdec.squeeze(0))
        pred_id = output.squeeze().argmax().item()
        if pred_id == vocabidx_y['<eos>']:
            break
        pred_y = vocablist_y[pred_id][0]
        pred.append(pred_y)
        y[0]=pred_id
        input=y
    return pred

## Train the model

In [41]:
def train():
  model = LSTM(vocablist_en, vocabidx_en, vocablist_vi, vocabidx_vi).to(DEVICE)
  optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
  for epoch in range(EPOCH):
    loss = 0
    step = 0
    for ben, bvi in train_data:
      ben = torch.tensor(ben, dtype=torch.int64).transpose(0,1).to(DEVICE) 
      bvi = torch.tensor(bvi, dtype=torch.int64).transpose(0,1).to(DEVICE)
      optimizer.zero_grad()
      batchloss = model((ben, bvi))
      batchloss.backward()
      optimizer.step() 
      loss = loss + batchloss.item()
      if step % 100 == 0:
        print("step:", step, "batch loss:", batchloss.item())
      step += 1
    print("epoch", epoch, ": loss", loss)
  torch.save(model.state_dict(), MODELNAME)

In [42]:
def test():
  total = 0
  correct = 0
  model = LSTM(vocablist_en, vocabidx_en, vocablist_vi, vocabidx_vi).to(DEVICE)
  model.load_state_dict(torch.load(MODELNAME))
  model.eval()
  ref = []
  pred = []
  for enprep, en, vi in test_data:
    input = torch.tensor([enprep], dtype=torch.int64).transpose(0, 1).to(DEVICE)
    # flat_inputs = torch.flatten(input)
    p=model.evaluate(input, vocablist_vi, vocabidx_vi)
    print("INPUT", en)
    print("REF", vi)
    print("MT", p)
    ref.append([vi])
    pred.append(p)
  bleu = torchtext.data.metrics.bleu_score(pred, ref)
  print("total:", len(test_data)) 
  print("bleu:", bleu)

In [ ]:
EPOCH = 20
train()

In [ ]:
test()

## Fine tune 

### epochs = 10, batch size = 128, embedding feature number = 500, dropout with p = 0.2, LR = 0.0001, hidden feature number of LSTM = 512.

total: 1268

bleu: 0.05002653860452189

### epochs = 20, batch size = 128, embedding feature number = 256, dropout with p = 0.1, LR = 0.0001, hidden feature number of LSTM = 516, add 1 a dense_output 

total: 1268

bleu: 0.06727238576000198

### epochs = 20, batch size = 128, embedding feature number = 500, dropout with p = 0.2, LR = 0.0001, hidden feature number of LSTM = 512.

total: 1268

bleu: 0.06942979589406871

### epochs = 40, batch size = 128, embedding feature number = 256, dropout with p = 0.1, LR = 0.0001, hidden feature number of LSTM = 516, add 1 a dense_output

total: 1268

bleu: 0.07413500685516883

### epochs = 20, batch size = 128, embedding feature number = 500, dropout with p = 0.2, LR = 0.001, hidden feature number of LSTM = 512.

total: 1268

bleu: 0.07790628280631366

### epochs = 20, batch size = 64, embedding feature number = 500, dropout with p = 0.1, LR = 0.001, hidden feature number of LSTM = 512, add 1 dense_output 

total: 1268

bleu: 0.09107447172513936

### epochs = 20, batch size = 64, embedding feature number = 500, dropout with p = 0.2, LR = 0.001, hidden feature number of LSTM = 516.

total: 1268

bleu: 0.09593219314336811

### epochs = 20, batch size = 64, embedding feature number = 500, dropout with p = 0.2, LR = 0.001, hidden feature number of LSTM = 512.

total: 1268

bleu: 0.10033195446730946